<h2>导入包</h2>

In [46]:
import logging
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

In [47]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [48]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"])
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"])

In [49]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"])
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"])
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [50]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")

<h2>特征工程</h2>

<h3>时间特征</h3>

In [51]:
df_train["年"] = df_train["时间"].dt.year
df_train["月"] = df_train["时间"].dt.month
df_train["日"] = df_train["时间"].dt.day
df_train["分"] = df_train["时间"].dt.minute // 15 + df_train["时间"].dt.hour * 4
df_train = df_train.drop(columns=["时间"])

df_test["年"] = df_test["时间"].dt.year
df_test["月"] = df_test["时间"].dt.month
df_test["日"] = df_test["时间"].dt.day
df_test["分"] = df_test["时间"].dt.minute // 15 + df_test["时间"].dt.hour * 4
df_test = df_test.drop(columns=["时间"])

<h2>训练模型</h2>

<h3>评测指标</h3>

In [52]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [53]:
params_lgb = {
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
}

In [54]:
model_lgb = dict()
kfold = KFold(n_splits=5, random_state=42, shuffle=True)
for site, df in df_train.groupby("光伏用户编号"):
    model_lgb[site] = []
    df = df.drop(columns=["光伏用户编号"]).astype(np.float32)
    y = df.pop("target")
    df["辐照强度（J/m2） - 1"] = df["辐照强度（J/m2）"].shift(1)
    df["辐照强度（J/m2） - 2"] = df["辐照强度（J/m2）"].shift(2)
    mse = 0
    for fold, (train_index, val_index) in enumerate(kfold.split(df, y)):
        logging.info(f'############ site: {site} fold: {fold} ###########')
        x_train, x_val, y_train, y_val = df.iloc[train_index], df.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
        trainset = Dataset(x_train, y_train)
        valset = Dataset(x_val, y_val)
        model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], num_boost_round=5000, callbacks=[lgb.early_stopping(100), lgb.log_evaluation(1000)])
        model.save_model("../models/lgb_%s_%d.txt" % (site, fold))
        model_lgb[site].append(model)
        val_pred = Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
        mse += mean_squared_error(y_val.fillna(0), val_pred)
    rmse = np.sqrt(mse)
    score = 1 / (1 + rmse)
    logging.info(f"--------------site: {site} 本地分数 {score}---------------")

2024-03-07 16:06:21,512 : INFO : ############ site: f1 fold: 0 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.00803111	valid_1's l2: 0.0191556
[2000]	training's l2: 0.00405073	valid_1's l2: 0.0181388
Early stopping, best iteration is:
[2481]	training's l2: 0.00310367	valid_1's l2: 0.0179793


2024-03-07 16:06:24,730 : INFO : ############ site: f1 fold: 1 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.00775778	valid_1's l2: 0.0225184
[2000]	training's l2: 0.00392952	valid_1's l2: 0.0212867
[3000]	training's l2: 0.00230164	valid_1's l2: 0.020864
[4000]	training's l2: 0.00145357	valid_1's l2: 0.0206752
Early stopping, best iteration is:
[3918]	training's l2: 0.00150467	valid_1's l2: 0.0206726


2024-03-07 16:06:29,649 : INFO : ############ site: f1 fold: 2 ###########


...

2024-03-07 16:09:21,152 : INFO : ############ site: f9 fold: 3 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.000488141	valid_1's l2: 0.00113801
[2000]	training's l2: 0.000252126	valid_1's l2: 0.00105856
[3000]	training's l2: 0.000152363	valid_1's l2: 0.00103845
[4000]	training's l2: 9.9089e-05	valid_1's l2: 0.00102983
[5000]	training's l2: 6.73442e-05	valid_1's l2: 0.001026
Did not meet early stopping. Best iteration is:
[5000]	training's l2: 6.73442e-05	valid_1's l2: 0.001026


2024-03-07 16:09:28,447 : INFO : ############ site: f9 fold: 4 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.000487253	valid_1's l2: 0.00116986
[2000]	training's l2: 0.000253374	valid_1's l2: 0.00109423
[3000]	training's l2: 0.000152495	valid_1's l2: 0.0010716
Early stopping, best iteration is:
[3763]	training's l2: 0.000108871	valid_1's l2: 0.00106339


2024-03-07 16:09:34,065 : INFO : --------------site: f9 本地分数 0.9322972820202412---------------


<h2>预测</h2>

In [55]:
for site, df in df_test.groupby("光伏用户编号"):
    df = df.drop(columns=["光伏用户编号"]).astype(np.float32)
    y = df.pop("target")
    df["辐照强度（J/m2） - 1"] = df["辐照强度（J/m2）"].shift(1)
    df["辐照强度（J/m2） - 2"] = df["辐照强度（J/m2）"].shift(2)
    y_pred = np.zeros((df.shape[0], ))
    for model in model_lgb[site]:
        y_pred += model.predict(df, num_iteration=model.best_iteration)
    y_pred = y_pred / kfold.n_splits
    df_test.loc[df_test["光伏用户编号"] == site, "target"] = y_pred

In [56]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

In [57]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [58]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)